In [34]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import math
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score
from scipy.stats import pearsonr
from tensorflow.keras.models import load_model
from joblib import load
from sklearn.metrics import auc
import joblib
import shutil
from collections import Counter
import random


In [35]:
model = tf.keras.models.load_model('h5/EXPERIMENT.h5')

# Generar gráficas con valores nuevos

Creación listas

In [36]:
# Lista radio: números del 10 al 52
radio = list(range(5, 53))

# Lista fracvol: tantos unos como números en la lista radio
fracvol = [1 for _ in range(len(radio))]

# Lista espesor: repite valores entre 100 y 120 hasta alcanzar la longitud de la lista radio
espesor = [random.randint(100, 110) for _ in range(len(radio))]


# Ahora las listas radio, fracvol, y espesor están listas para ser usadas
longitud_de_onda = list(range(301, 901))

Checando existencia de la carpeta

In [37]:
# Ruta del directorio
dir_path = 'graficas_nuevas'

# Si el directorio ya existe, eliminarlo
if os.path.exists(dir_path):
    shutil.rmtree(dir_path)

# Crear el directorio
os.makedirs(dir_path)

In [39]:
# Asegurar que la carpeta existe o crearla
carpeta = dir_path
if not os.path.exists(carpeta):
    os.mkdir(carpeta)

# Asumiendo que todos los arrays (fracvol, radio, espesor) tienen la misma longitud:
num_elementos = len(fracvol)

scaler_X = load('minmax_scaler_X.pkl')

for i in range(num_elementos):
    for j in range(num_elementos):
        data = {
            "Longitud de onda": longitud_de_onda,
            "Fracvol": [fracvol[i]] * len(longitud_de_onda),
            "Radio (nm)": [radio[i]] * len(longitud_de_onda),
            "Espesor de la película (nm)": [espesor[j]] * len(longitud_de_onda)
        }
        
        df = pd.DataFrame(data)
        x = df.values
        x_norm = scaler_X.transform(x)
        predictions = model.predict(x_norm)

        df["Índice de absorción"] = predictions

        # Filtrar datos entre 400 a 700 nm
        df_filtrado = df[(df['Longitud de onda'] >= 400) & (df['Longitud de onda'] <= 700)]
        
        # Calcular AUC
        valor_auc = auc(df_filtrado['Longitud de onda'], df_filtrado['Índice de absorción'])

        columnas_seleccionadas = ['Longitud de onda', 'Índice de absorción']
        df_seleccionado = df[columnas_seleccionadas]
        titulo = f'fracción volumen: {fracvol[i]:.2f}, radio: {radio[i]:.2f}, espesor: {espesor[j]:.2f}, AUC: {valor_auc:.2f}'

        # Crear la gráfica de dispersión
        plt.figure(figsize=(10, 6))
        plt.plot(df_seleccionado['Longitud de onda'], df_seleccionado['Índice de absorción'], color='blue')
        plt.xlabel('Longitud de onda')
        plt.ylabel('Índice de absorción')
        plt.title(titulo)
        plt.grid(True)
        
        # Guardar la gráfica como imagen en la carpeta especificada
        nombre_archivo = os.path.join(carpeta, f"grafica_{fracvol[i]}_{radio[i]}_{espesor[j]}.png")
        plt.savefig(nombre_archivo)
        plt.close()

19/19 [==============================] - 0s 767us/step
